# HW 2- group 5

In [1]:
# imports
import os
import sys
import pyspark
from pyspark import SparkContext, SparkConf
import pandas as pd

## create new folders for the data and load the data

In [2]:
%%bash
hdfs dfs -mkdir /user/stud27/HW2_data

In [3]:
%%bash
hdfs dfs -mkdir /user/stud27/HW2_data/input

In [4]:
%%bash
hdfs dfs -ls /user/stud27/HW2_data

Found 1 items
drwxr-xr-x   - stud27 stud27          0 2021-06-16 12:02 /user/stud27/HW2_data/input


In [5]:
%%bash
hdfs dfs -put /home/stud27/data_hw2 /user/stud27/HW2_data/input

In [6]:
%%bash
hdfs dfs -ls /user/stud27/HW2_data/input/data_hw2

Found 3 items
-rw-r--r--   3 stud27 stud27        387 2021-06-16 12:02 /user/stud27/HW2_data/input/data_hw2/countries.txt
-rw-r--r--   3 stud27 stud27    1491520 2021-06-16 12:02 /user/stud27/HW2_data/input/data_hw2/microsoft-com.data
-rw-r--r--   3 stud27 stud27       3629 2021-06-16 12:02 /user/stud27/HW2_data/input/data_hw2/microsoft-com.info


## Question 1

In [7]:
data_rdd = sc.textFile("hdfs:/user/stud27/HW2_data/input/data_hw2/microsoft-com.data")
country_rdd = sc.textFile("hdfs:/user/stud27/HW2_data/input/data_hw2/countries.txt")

In [8]:
%%bash
hdfs dfs -cat /user/stud27/HW2_data/input/data_hw2/microsoft-com.data | head -n 5

A,1287,1,"International AutoRoute","/autoroute"
A,1288,1,"library","/library"
A,1289,1,"Master Chef Product Information","/masterchef"
A,1297,1,"Central America","/centroam"
A,1215,1,"For Developers Only Info","/developer"


cat: Unable to write to output stream.


In [9]:
%%bash
hdfs dfs -cat /user/stud27/HW2_data/input/data_hw2/microsoft-com.info

1. Title: "Anonymous web data from www.microsoft.com"

2. Source Information
   -- Creators: Jack S. Breese, David Heckerman, Carl M. Kadie
     -- Microsoft Research, Redmond WA, 98052-6399, USA
     -- breese@microsoft.com, heckerma@microsoft.com, carlk@microsoft.com
   -- Donor: Breese, Heckerman, & Kadie
   -- Date: November 1998
 
3. Past Usage:
   -- J. Breese, D. Heckerman., C. Kadie _Empirical Analysis of
      Predictive Algorithms for Collaborative Filtering_ Proceedings
      of the Fourteenth Conference on Uncertainty in Artificial Intelligence,
      Madison, WI, July, 1998. 
	Also, expanded as Microsoft Research Technical Report MSR-TR-98-12,
	The papers are available on-line at:
             http://research.microsoft.com/users/breese/cfalgs.html
      -- Results: Used to compare various memory-based and model-based collobative
         filtering.
    - Predicted attribute: Goal was to predict what areas of the web site a
         user visited based on data on what other 

## question 2

In [10]:
# filter & map the data according to the format specified in the info files (for type 'A'- attributes)
rdd_attributes = data_rdd.filter(lambda line: line[0] == 'A')\
                .map(lambda line: line.split(','))\
                .map(lambda line:[str(line[1]),str(line[3]).replace('"',''),str(line[4]).replace('"','')])

In [11]:
print rdd_attributes.take(5)

[['1287', 'International AutoRoute', '/autoroute'], ['1288', 'library', '/library'], ['1289', 'Master Chef Product Information', '/masterchef'], ['1297', 'Central America', '/centroam'], ['1215', 'For Developers Only Info', '/developer']]


In [12]:
# filter & map the data according to the format specified in the info files (for type 'V'- votes)
rdd_votes = data_rdd.filter(lambda line: line[0] == 'V')\
                .map(lambda line: line.split(','))\
                .map(lambda line:[str(line[2]),str(line[1])])

In [13]:
print rdd_votes.take(3)

[['1000', '10001'], ['1001', '10001'], ['1002', '10001']]


## question 3

In [14]:
print country_rdd.take(3)

[u'Argentina', u'Australia', u'Belgium']


In [15]:
# create a list with all the countries
countries_lst = country_rdd.collect()

In [16]:
# first, filter the data that are is in the counrties list
filter_attributes_by_countries = rdd_attributes.filter(lambda r: r[1] in countries_lst)
# join the 2 rdd so we get only the user that visited at least 1 country from the list
filter_users = filter_attributes_by_countries.join(rdd_votes)
# leave only the user's id
users = filter_users.map(lambda line : line[1][1]).distinct()
# show the 10 first users
print (users.take(10))

['36585', '40557', '20911', '40720', '35548', '14196', '34279', '33731', '28995', '16424']


## question 4

In [17]:
# use map-reduce to count the number of user for each country.
# from the map we get (country_name,1) and in the reduce sum the 1's for each country
country_count_users = filter_users.map(lambda line : [line[1][0],1]).reduceByKey(lambda v1,v2: v1+v2)

In [18]:
# get a list from the results we got above
count_country_users_lst = country_count_users.collect()
# get only the countries from the results we got above
countries = country_count_users.map(lambda r : r[0]).distinct().collect()
# for each country check if the country not in the countries with at least 1 user
for country in countries_lst:
    if country not in countries:
        # add this country eith 0 in the count
        count_country_users_lst.append((country,0)) 
# convert the list with all the countries to rdd
count_countries = sc.parallelize(count_country_users_lst)

In [19]:
# order the data in DESC order by the count to get the top 5 countries
top_5_count_countries = count_countries.takeOrdered(5, lambda x: -x[1])
# convert to data frame to display the report
display(pd.DataFrame(top_5_count_countries,columns=['country','number_of_users']))

,country,number_of_users
0,Jakarta,670
1,Germany,372
2,Sweden,258
3,Taiwan,204
4,Spain,191


## Question 5

In [20]:
%%bash
hdfs dfs -put /home/stud27/data_hw2 /user/stud27/HW2_data/output

In [21]:
#convert rdd to the following format: country:number_of_users and save in HDFS
count_countries.map(lambda line:"{}:{}".format(line[0],line[1])).saveAsTextFile("hdfs:/user/stud27/HW2_data/output/country_count")

## Question 6

In [22]:
# use map reduce to get for each user the number of pages.
# the first map reduce: from the map we got (user_id,1) and in the reduce sum the 1's for each user
# the second map reduce: get the output from above and we got from the map ('id', (the sum of 1's,1))
# and in the reduce we sum the total number of sums and the number of times we did it.
# finally we caculate the average
pages_per_user = rdd_votes.map(lambda line: (line[1],1))\
                          .reduceByKey(lambda v1,v2: v1+v2)\
                          .map(lambda line: ('id', (line[1], 1)))\
                          .reduceByKey(lambda v1,v2: (float(v1[0])+float(v2[0]),v1[1]+v2[1]))\
                          .mapValues(lambda v: v[0] / v[1]) 

print 'The average is:',pages_per_user.first()[1]          

The average is: 3.01592736388


## Question 7

In [23]:
# use map reduce to get the page id with the maximm visits
# the first map reduce: from the map we got (page_id,1) and in the reduce sum the 1's for each page
# the second map reduce: get the output from above and we got from the map ('id', (page_id,count_visits))
# and in the reduce we checks each time which page id has the biggest count.
page_max_visits = rdd_votes.map(lambda line: (line[0],1))\
                    .reduceByKey(lambda v1,v2: v1+v2)\
                    .map(lambda line: ('id',line))\
                    .reduceByKey(lambda v1,v2: v1 if v1[1]>v2[1] else v2)
final_data = page_max_visits.collect()[0][1]
print 'The id for the page with the maximum visits is',final_data[0], 'and the number of visits is', final_data[1]

The id for the page with the maximum visits is 1008 and the number of visits is 10836


## remove files from HDFS

In [24]:
%%bash
hdfs dfs -rm -r  /user/stud27/HW2_data

21/06/16 12:02:48 INFO fs.TrashPolicyDefault: Moved: 'hdfs://bdl0.eng.tau.ac.il:8020/user/stud27/HW2_data' to trash at: hdfs://bdl0.eng.tau.ac.il:8020/user/stud27/.Trash/Current/user/stud27/HW2_data


In [25]:
%%bash
hdfs dfs -ls  /user/stud27/

Found 1 items
drwx------   - stud27 stud27          0 2021-06-16 12:02 /user/stud27/.Trash
